In [ ]:
pip install lightkurve

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 KB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 KB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 KB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 KB 5.6 MB/s eta 0:00:00
  Created wheel for fbpca: filename=fbpca-1.0-py3-none-any.whl size=11373 sha256=26471727f33dcd06081096edf69d2bb749bf6444fd332a0446eefbc25a4e2eb1
  Stored in directory: /root/.cache/pip/wheels/f5/60/60/28df6c25f4d22b73d1a2b1c7e4842a5e2178e35e47b62e8e9a
  Created wheel for memoization: filename=m

In [ ]:
pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import lightkurve as lk
import astropy

from astropy.io import fits
from google.colab import files
import csv
import sys
from timeit import default_timer as timer
from sklearn.preprocessing import normalize

import numpy as np

def weightSoma(fluxErr):
    return np.power(np.sum(np.power(fluxErr,-2)),-1)

def rValue(i1,i2,weight):
    return np.sum(np.fromiter((weight[i] for i in range(i1,i2)), dtype=float))

def sValue(i1,i2,weight,fluxo):
    return np.sum(np.fromiter((weight[i]*fluxo[i] for i in range(i1,i2)), dtype=float))

def dValue(lisWeight,fluxo,r,s):
    soma = np.sum(np.fromiter((lisWeight[i]*fluxo[i]**2 for i in range(len(lisWeight))), dtype=float))
    return (soma - (s**2)/(r*(1-r)))

#search_result = lk.search_lightcurve('Kepler-37', author='Kepler', cadence='long')
# Download all available Kepler light curves
#c_collection = search_result.download_all()
#for x in range(1) :
#  dataf = lc_collection[x]
#  lc = dataf.remove_nans().remove_outliers()
#  lc.scatter()

"""
with fits.open('/content/mockcurve_1757_p276.0.fits') as hdul:
  hdul.info()
  #hdr = hdul[0].header
  #print(repr(hdr))
  hdul[1] = 'Selin'
  table_data =  hdul[1].data
  cols = hdul[1].columns
  cols.info()
 
  # use to_fits to write lc in fits extension by file system
  lc.to_fits(path=(str(x) + '.fits'), overwrite=True)
  #hdu = fits.open((str(x) + '.fits'))
  #hdu.info()
  #table_data =  hdu[1].data
  
  #cols = hdu[1].columns
  #cols.info()

  print(dataf)
  #dataf.info()
  #print(dataf)
  #table_data =  dataf[1].data
  #cols = dataf[1].columns
  #cols.info()
  time = dataf['time']
  flux = dataf['sap_flux']
  fluxerr = dataf['sap_flux_err']
  with open((str(x) + '.csv'), 'w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(["Time", "Flux", "FluxErr"])
    for i in range(len(time)) :
      writer.writerow([time[i], flux[i], fluxerr[i]])
"""

def myBls(flux, fluxerr):
  lisWeight = []
  somaW = weightSoma(fluxerr)
  
  for i1 in range(len(flux)):
    wi = somaW*(fluxerr[i1]**(-2))
    lisWeight.append(wi)

  min_d = sys.float_info.max
  min_i1 = -1
  min_i2 = -1
  for i1 in range(15,len(flux)):
    for i2 in range(i1+1,len(flux)):
      r = rValue(i1,i2,lisWeight)
      s = sValue(i1,i2,lisWeight,flux)
      d = dValue(lisWeight,flux,r,s)
      if d<min_d :
        min_d=d
        min_i1=i1
        min_i2=i2
        #print(f"min found: s {s}, r = {r} , d = {d}")
  print(f"FINAL D: {d}, with i1: {min_i1} , i2: {min_i2} ")
  return min_i1,min_i2 

def main():
  for y in range(60,129) :
    search_result = lk.search_lightcurve(('Kepler-' + str(y)), author='Kepler', cadence='long')
    # Download all available Kepler light curves
    lc_collection = search_result.download_all()
    for x in lc_collection :
      dataf = x
      #lc = LightCurve(time=[1, 2, 3, 4, 5], flux=[1, 1000, 1, -1000, 1])
      lc_clean = dataf.remove_outliers(sigma=1)
      time = lc_clean['time']
      if(len(time) > 15000 and len(time) < 20000) :
        print(dataf.filename)
        print(len(time))
        flux =lc_clean['sap_flux']
        fluxerr = lc_clean['sap_flux_err']
        ntime = np.array(time)
        nflux = np.array(flux) 
        nfluxerr = np.array(fluxerr) 
        lc_clean.to_fits(path="/content/drive/MyDrive/mock_curves/"+"mockcurve" + str(ntime.size)+".fits", overwrite=True)
  

  """
  col1 = fits.Column(name='TIME', format='D', array=(ntime))
  print(col1)
  col2 = fits.Column(name='FLUX', format='D', array=(nflux))
  col3 = fits.Column(name='FLUX_ERR', format='D', array=(nfluxerr))
  coldefs = fits.ColDefs([col1, col2, col3])
  hdu = fits.BinTableHDU.from_columns(coldefs)
  hdu.name = "LIGHTCURVE"

  prihdr = fits.Header()
  prihdu = fits.PrimaryHDU(header=prihdr)
  prihdu.name = "PRIMARY"

  thdulist = fits.HDUList([prihdu, hdu])
  thdulist.writeto("/content/selin/smallestLC"+str(time.size)+".fits")
  """

"""
  for x in range(0,1) :
    dataf = lc_collection[x]
    #dataf.info()
    #cols = dataf[1].columns
    print(f"Number of Lines in File: {len(dataf)}")
    #cols.info()
    time = np.asarray(dataf['time'])
    flux = np.asarray(dataf['sap_flux']) 
    fluxerr = np.asarray(dataf['sap_flux_err'])
    #print(f"datafile: {d}, with i1: {min_i1} , i2: {min_i2} ")

    #clean nans 
    flux = flux[np.logical_not(np.isnan(flux))]
    fluxerr = fluxerr[np.logical_not(np.isnan(fluxerr))]
    #normalize
    flux = flux / np.linalg.norm(flux)
    fluxerr = fluxerr / np.linalg.norm(fluxerr)
    #clean 0.0s
    #flux = flux[np.logical_not(flux == 10^-13)]
    #fluxerr = fluxerr[np.logical_not(fluxerr == 10^-13)]
    print(f"Remaining Lines in File after Cleansing: {len(flux)}")
  
    start = timer()
    i1,i2 = myBls(flux,fluxerr )
    end = timer()
    print(f"Execution Time: {end-start} with Period: {time[i2]-time[i1]}")
    #break
    
    with open((str(x) + '.csv'), 'w', newline='') as file:
      writer = csv.writer(file)

      writer.writerow(["Time", "Flux", "FluxErr"])
      for i in range(len(time)) :
        writer.writerow([time[i], flux[i], fluxerr[i]])
  """
main()

Could not resolve Kepler-72 to a sky position.
ERROR:lightkurve.search:Could not resolve Kepler-72 to a sky position.
/usr/local/lib/python3.9/dist-packages/lightkurve/search.py:485: LightkurveWarning: Cannot download from an empty search result.
  warnings.warn(


TypeError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
